# GPT 

На прошлой паре мы разбирали транформер в целом и попробовали дообучать BERT для задачи классификации. На этом посмотрим на другую популярную транформерную модель -  GPT (Generative pretrained transformer).

## Чем BERT и GPT похожи?
- Обе модели основаны на транформерных блоках с механизмом внимания и позиционным кодированием. Есть небольшие отличия в некоторых частях моделей (в gpt, например используется sparse attention), но они не принципиальные.

- В обоих моделях используется BPE токенизация. 
- Обе модели решают задачу языкового моделирования.
- В обеих моделях используется только self-attention. 

## Чем отличаются BERT и GPT?
- BERT - это энкодер модель. А GPT - это декодер модель.
- BERT обучается на задачах Мasked language modelling (MLM) и Next Sentence Prediction (NSP). Это искусственные self supervised задачи: в MLM модель должна заполнить пропуски в тексте (например, "Я [MASK] в [MASK] один." -> "Я пошел в кино один"), а NSP - определить являются ли два предложения соседними. А GPT - это авторегрессионная модель, которая учится продолжать данную ей последовательностью ("Я пошел.." -> "Я пошел в" -> "Я пошел в кино" -> "Я пошел в кино один")
- BERT смотрит на входной текст целиком, а в GPT attention для каждого отдельного токена расчитывается только по предудущим токенам.  
- В GPT (особенно в последней версии огромных размеров) целевой задачей является именно генерация текста и поэтому предобученную модель либо вообще никак не дообучают, либо дообучают на генерацию нужных текстов (а не конкретную задачу вроде классификации). BERT используется в основном только для эмбеддингов и дообучения на конкретные задачи (классификация), заполнение пропусков в практических задачах практически бесполезно. 

### Главная особенность GPT
Предобученную GPT можно использовать для решения целевой задачи (той же классификации) сразу из коробки, без дообучения, через генерацию текста. Нужно только сформулировать входную последовательность подходящим образом. Например, для определения класса текста можно подать в модель нужный текст и добавить в конце "Тема этого текста: ", на что модель выдаст продолжение "Новости" или "Спорт". Такой подход называется zero-shot learning. 
Еще есть few-shot learning, когда в модель подается что-то вроде "2+2=4, 1+3=4, 1+2= ", то есть модели показывается несколько примеров того, что от нее хотят и просят анологично продолжить последний пример. В этом случае также не происходит дообучения модели (веса не обновляются), поэтому на каждом запуске в модель нужно подавать нужный контекст. Такие контексты называются затравками (prompts). Подбирать их может быть достаточно сложно, даже есть такой термин "prompt engineering". (Есть способ обойти ручную подброку затравок, который называется prompt tuning. Про него можно почитать вот тут - https://habr.com/ru/company/yandex/blog/588214/) 

Также важная характеристика GPT моделей - их огромный размер. Самая большая модель GPT-3 имеет 175 млрд параметров, и она даже не выложена в открытый доступ, а предоставляется через API (https://openai.com/api/). Есть варианты поменьше, в том числе и для русского (RuGPT-3 от Сбера). Их можно попробовать в коллабе через huggingface. Давайте попробуем!



In [1]:
%pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 32.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.5/770.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 54.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers.utils import logging
logging.set_verbosity(40)

In [5]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

# Загружаем модель ruGPT от сбера
model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

In [6]:
# prompt engineering 
text = "Вопрос: 'Сколько будет 2+2?'\n Ответ: " # работает
# text = "Вопрос: 'Сколько будет 3+3?'\n Ответ: 6. Вопрос: 'Сколько будет 1+9?'\n Ответ: 10. Вопрос: 'Сколько будет 4+2?'\n Ответ:" # не очень работает
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
out = model.generate(input_ids, do_sample=False) 

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Вопрос: 'Сколько будет 2+2?'
 Ответ: 
2+2=4


## Методы генерации текста
В вводном семинаре про языковое моделирование мы уже пробовали генерировать текст и там мы сталкивались с вопросом - а как выбирать конкретный токен по вероятностям, которые выдает модель? Если просто использовать .argmax(), то генерация будет слишком детерменированной и будет много повторов. Обычно это не то, что нам нужно. GPT может применяться, например, для генерации ответов в чатботе и однобразность ответов - сильный минус.

На семинаре про базовое языковое моделирование мы решали это случайным семплированием из полученного распределения, но на практике используют немного другие подходы. Давайте на них посмотрим.

In [7]:
# возьмем какой-нибудь текст
text = 'За окном дождь. Холодный и противный. Хочется'
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)

### Argmax
Мы уже выше попробовали аргмакс, но давайте проверим его и на новом тексте, чтобы потом было с чем сравнивать

In [8]:
# если повторить запуск результат не изменится
# но тут уже нет зацикливания потому что модель смотрит дальше чем два токена в прошлое
out = model.generate(input_ids, do_sample=False, max_length=50)


generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


За окном дождь. Холодный и противный. Хочется плакать.

— Я не могу, — говорит она. — Я не могу.

— Почему?

— Потому что я не могу.




### Сэмплирование

Чтобы на каждом шаге выбирать случайное слово с учетом распределения, нужно просто поменять do_sample на True. Так результат будет меняться каждый раз

In [9]:
out = model.generate(input_ids, do_sample=True,  
                     top_k=0,  # про это параметр ниже
                     max_length=50)


generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


За окном дождь. Холодный и противный. Хочется закрыть глаза, но я боюсь об этом думать. Я боюсь, что из-за этого дождя у них будут трудности… Я боюсь, что из-за моей неловкости они откажут мне в


Но случайный выбор может быть слишком случайным, так как иногда будут выбираться совсем маловероятные слова. Есть сразу несколько параметров, которые помогают контролировать случайность.

### top_k

Этот параметр ограничивает количество слов, из которых мы семплируем. 10 - означает, что мы выбирает только из 10 самых вероятных слов.
В ячейке выше мы поставили top_k = 0, потому что по умолчанию он стоит 50, а нам нужно было попробовать без него. 


Чем больше top_k тем более случайный результат мы получим, но слишком низкие top_k может плохо сказаться на разнообразности.

In [14]:
for top_k in [1,3,10, 30, 100]:

    out = model.generate(input_ids, do_sample=True,  
                     top_k=top_k,  # про это параметр ниже
                     max_length=50)


    generated_text = list(map(tokenizer.decode, out))[0]
    print("### text with top_k - ", top_k)
    print(generated_text)
    print()

### text with top_k -  1
За окном дождь. Холодный и противный. Хочется плакать.

— Я не могу, — говорит она. — Я не могу.

— Почему?

— Потому что я не могу.



### text with top_k -  3
За окном дождь. Холодный и противный. Хочется закрыть глаза. Закрыть глаза и не видеть. Не видеть. Не видеть. Не видеть.

Я не хочу видеть, как он уходит. Не хочу. Не хочу. Не хочу.

### text with top_k -  10
За окном дождь. Холодный и противный. Хочется плакать, но слез нет.

— Что случилось? — спросила я.

Мама, конечно же, ответила: «Следи за своим языком». А потом она ушла

### text with top_k -  30
За окном дождь. Холодный и противный. Хочется закрыть окна, чтобы хоть немного согреться и отогреться. Дождь, дождь, дождь. Спать.
И, вдруг, мне хочется заплакать.
Как только я это представляю, на

### text with top_k -  100
За окном дождь. Холодный и противный. Хочется закрыть глаза и не видеть его. На улице как в лесу идет снег. За пару месяцев почти все листья облетели. В этом году я была без очков. З

### Сэмплирование с Температурой
Еще случайность можно контролировать с помощью параметра, который называется температура. Температура изменяет распределение - при низком значении температуры вероятности переносятся на от низких значений к высоким (распределение заостряется), а при высоком - вероятности переносятся от высоких значений к низким (распределение сглаживается).

Нулевая температура означает, что мы на каждом шаге просто выбираем по argmax(), а очень большая температура будет приводить к полному рандому. Под конкретную задачу температуру нужно подбирать отдельно, можно начать с 0 и постепенно увеличивать, смотря на получаемое разнобразие.

(температурой это называется потому что формула взята из физических уравнений, где этот параметр действительно отвечает за температуру)

In [21]:
for temp in [0.001, 0.1, 0.2, 0.5, 0.7, 1., 5.]:

    out = model.generate(input_ids, do_sample=True,  
                     top_k=0, 
                     temperature=temp,
                     max_length=50)


    generated_text = list(map(tokenizer.decode, out))[0]
    print("### text with temp - ", temp)
    print(generated_text)
    print()

### text with temp -  0.001
За окном дождь. Холодный и противный. Хочется плакать.

— Я не могу, — говорит она. — Я не могу.

— Почему?

— Потому что я не могу.



### text with temp -  0.1
За окном дождь. Холодный и противный. Хочется плакать.

— Я не могу, — говорит она. — Я не могу.

— Почему?

— Потому что я не могу.



### text with temp -  0.2
За окном дождь. Холодный и противный. Хочется закрыть глаза и закрыть уши. Но я не могу. Я не могу.

Я не могу.

Я не могу.

Я не могу.

Я не могу.

### text with temp -  0.5
За окном дождь. Холодный и противный. Хочется, чтобы на улице было тепло.

— Жаль, что ты не можешь оставить нас, — говорит отец.

— Я не могу, — отвечаю я.

### text with temp -  0.7
За окном дождь. Холодный и противный. Хочется надеть что-нибудь потеплее, да плюнуть на все и уйти. Хотя если бы было теплее, я бы с удовольствием осталась. Дождь в такую погоду - это что-то новое. Я

### text with temp -  1.0
За окном дождь. Холодный и противный. Хочется жить. Живу. Есть

Видно, что чем больше температура, тем случайнее текст


Эти параметры можно совмещать, чтобы точнее найти границу между разнообразием и рандомом

In [25]:
out = model.generate(input_ids, 
                     do_sample=True,
                     temperature=0.9,
                     top_k=10,
                     max_length=50,
                    )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


За окном дождь. Холодный и противный. Хочется домой.

В комнате на столе записка.

«Я уезжаю в Америку.

Я вернусь через несколько недель.

С любовью, мама.

    Р


Температура большая, но текст все равно получается более менее тематически связанный

### Beam Search

У подходов выше есть недостаток - на каждом шаге выбирается только 1 слово и этот выбор нельзя изменить, поэтому 1 неверно выбраное слово можно испортить весь текст и это сложно проконтролировать температурой и топ-к.

С этим может помочь beam-search (поиск пучком). В нем по сути происходит одновременная генерация нескольких текстов параллельно и в конце выбирается текст с наибольшей общей вероятностью. Генерировать все возможные варианты невозможно технически (потому что количество вариаций растет очень быстро), поэтому в beam search варианты отсеиваются на каждом шаге так, чтобы количество текущих вариантов было не больше N. Этот параметр N (размер пучка, beam size) настраивается, но поставить его слишком большим не получится, т.к. опять же будет слишком много комбинаций и это увеличит время генерации. 

In [28]:
# beam search уже реализован в hg поэтому нужно только задать параметр num_beams
out = model.generate(input_ids, do_sample=True, num_beams=5, top_k=0, max_length=60)

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text.replace('<s>', ' '))


За окном дождь. Холодный и противный. Хочется залезть под одеяло и не вылезать до утра.

— Ты что, заболел?

— Нет.

— Что с тобой?

— Ничего.

— Может, врача вызвать?


Но это тоже не очень сильно помогает

# Дообучение GPT

Чтобы генерировать тексты в нужном стиле/домене, можно дообучить GPT на своем корпусе. Давайте попробуем это сделать

In [29]:
# возьмем модель поменьше, так как дообучение это обновление весов
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path, use_cache=False).to(DEVICE)


Для наглядности результата возьмем начало Евгения Онегина

In [36]:
text = """
«Мой дядя самых честных правил,
Когда не в шутку занемог,
Он уважать себя заставил
И лучше выдумать не мог.
Его пример другим наука;
Но, боже мой, какая скука
С больным сидеть и день и ночь,
Не отходя ни шагу прочь!
Какое низкое коварство
Полуживого забавлять,
Ему подушки поправлять,
Печально подносить лекарство,
Вздыхать и думать про себя:
Когда же черт возьмет тебя!»

Так думал молодой повеса,
Летя в пыли на почтовых,
Всевышней волею Зевеса
Наследник всех своих родных. —
Друзья Людмилы и Руслана!
С героем моего романа
Без предисловий, сей же час
Позвольте познакомить вас:
Онегин, добрый мой приятель,
Родился на брегах Невы,
Где, может быть, родились вы
Или блистали, мой читатель;
Там некогда гулял и я:
Но вреден север для меня.

Служив отлично-благородно,
Долгами жил его отец,
Давал три бала ежегодно
И промотался наконец.
Судьба Евгения хранила:
Сперва Madame за ним ходила,
Потом Monsieur ее сменил;
Ребенок был резов, но мил.
Monsieur l’Abbe€, француз убогой,
Чтоб не измучилось дитя,
Учил его всему шутя,
Не докучал моралью строгой,
Слегка за шалости бранил
И в Летний сад гулять водил.

Когда же юности мятежной
Пришла Евгению пора,
Пора надежд и грусти нежной,
Monsieur прогнали со двора.
Вот мой Онегин на свободе;
Острижен по последней моде;
Как dandy лондонский одет —
И наконец увидел свет.
Он по-французски совершенно
Мог изъясняться и писал;
Легко мазурку танцевал
И кланялся непринужденно;
Чего ж вам больше? Свет решил,
Что он умен и очень мил.

Мы все учились понемногу
Чему-нибудь и как-нибудь,
Так воспитаньем, слава богу,
У нас немудрено блеснуть.
Онегин был, по мненью многих
(Судей решительных и строгих),
Ученый малый, но педант.
Имел он счастливый талант
Без принужденья в разговоре
Коснуться до всего слегка,
С ученым видом знатока
Хранить молчанье в важном споре
И возбуждать улыбку дам
Огнем нежданных эпиграмм.

Латынь из моды вышла ныне:
Так, если правду вам сказать,
Он знал довольно по-латыни,
Чтоб эпиграфы разбирать,
Потолковать об Ювенале,
В конце письма поставить vale,
Да помнил, хоть не без греха,
Из Энеиды два стиха.
Он рыться не имел охоты
В хронологической пыли
Бытописания земли;
Но дней минувших анекдоты,
От Ромула до наших дней,
Хранил он в памяти своей.

Высокой страсти не имея
Для звуков жизни не щадить,
Не мог он ямба от хорея,
Как мы ни бились, отличить.
Бранил Гомера, Феокрита;
Зато читал Адама Смита
И был глубокий эконом,
То есть умел судить о том,
Как государство богатеет,
И чем живет, и почему
Не нужно золота ему,
Когда простой продукт имеет.
Отец понять его не мог
И земли отдавал в залог.

Всего, что знал еще Евгений,
Пересказать мне недосуг;
Но в чем он истинный был гений,
Что знал он тверже всех наук,
Что было для него измлада
И труд, и мука, и отрада,
Что занимало целый день
Его тоскующую лень, —
Была наука страсти нежной,
Которую воспел Назон,
За что страдальцем кончил он
Свой век блестящий и мятежный
В Молдавии, в глуши степей,
Вдали Италии своей.


Как рано мог он лицемерить,
Таить надежду, ревновать,
Разуверять, заставить верить,
Казаться мрачным, изнывать,
Являться гордым и послушным,
Внимательным иль равнодушным!
Как томно был он молчалив,
Как пламенно красноречив,
В сердечных письмах как небрежен!
Одним дыша, одно любя,
Как он умел забыть себя!
Как взор его был быстр и нежен,
Стыдлив и дерзок, а порой
Блистал послушною слезой!

Как он умел казаться новым,
Шутя невинность изумлять,
Пугать отчаяньем готовым,
Приятной лестью забавлять,
Ловить минуту умиленья,
Невинных лет предубежденья
Умом и страстью побеждать,
Невольной ласки ожидать,
Молить и требовать признанья,
Подслушать сердца первый звук,
Преследовать любовь и вдруг
Добиться тайного свиданья…
И после ей наедине
Давать уроки в тишине!


Как рано мог уж он тревожить
Сердца кокеток записных!
Когда ж хотелось уничтожить
Ему соперников своих,
Как он язвительно злословил!
Какие сети им готовил!
Но вы, блаженные мужья,
С ним оставались вы друзья:
Его ласкал супруг лукавый,
Фобласа давний ученик,
И недоверчивый старик,
И рогоносец величавый,
Всегда довольный сам собой,
Своим обедом и женой.


"""

В huggingface уже реализованы все инструменты для дообучения. Даже если вы не знакомы с pytorch разобраться будет не сложно

In [37]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# Сохраним обучающие данные в .txt файл 
train_path = 'train_dataset.txt'
with open(train_path, "w") as f:
    f.write(text)

# Создание датасета
train_dataset = TextDataset( tokenizer=tokenizer,file_path=train_path,block_size=64, 
                            overwrite_cache=True)
  
# специальный класс который будет подавать в модель данные в нужном ей виде
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Creating features from dataset file at 
Saving features into cached file cached_lm_GPT2Tokenizer_64_train_dataset.txt [took 0.000 s]


## Обучение
Через класс Trainer реализовано все обучение, останется только запустить фит (точнее трейн в случае hg)

In [38]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments( 
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,  
    gradient_accumulation_steps=16, 
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [39]:
trainer.train()

***** Running training *****
  Num examples = 19
  Num Epochs = 100
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 16
  Total optimization steps = 100
  Number of trainable parameters = 125231616


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=100, training_loss=0.03742799758911133, metrics={'train_runtime': 9.0992, 'train_samples_per_second': 208.81, 'train_steps_per_second': 10.99, 'total_flos': 62056857600000.0, 'train_loss': 0.03742799758911133, 'epoch': 100.0})

In [41]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=0.8,
                        top_k=50,
                        max_length=300,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1",
  "use_cache": false
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Дождь идет 
На Дон, 
На Дон его зовет. —
Друзья мои!
С героем моего романа
Умоляю вас,
Не щадите меня в минуту грусти,
Когда соловей поет,
Таит надежду в сердечных письмах,
Внимательно читает дневник,
Перепечатывает его вновь,
Переводит на французский язык,
Переводит на немецкий язык,
Переводит на русский язык
И наконец, в конце письма,
Дает вам честное слово,
Что он совершенно
Совместимый продукт
Добра и лжи.
Онегин на свободе;
Но где он теперь?
Где брат его Евгений?
Как он несчастен!
Как он страдает!
Как он боится всего,
Что мог бы сделать для России.
Как он был груб,
Когда писал ей!
Как он ревновал ее!
Как он был раздражителен!
Как он был жесток!
Как он был резок!
Как он был упрям!
Как он лицемерно подло
Ловил минуту за минутой
И наконец находил ей объяснение!
Как он был неумолим!
Как он ревновал ее!
Как он был жесток!
Как он был жесток!
Как он был жесток!
Как он был жесток!
Как он был жесток!
Как он был жесток!
Как он был жесток!
Как он был жесток!



Попробуем beam search

In [50]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5, top_k=50,
                        max_length=300,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1",
  "use_cache": false
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Дождь идет 
С севера на юг,
Вдали Италии своей.


Как рано мог уж он тревожить
Сердца кокеток записных!
Как рано мог уж он тревожить
Сердца кокеток записных!
Как рано мог уж он тревожить
Сердца кокеток записных!
Как рано мог уж он тревожить
Сердца кокеток записных!
Как рано мог уж он тревожить
Сердца кокеток записных!
Как рано мог уж он тревожить
Сердца кокеток записных!
Как рано мог уж он тревожить
Сердца кокеток записных!
Как рано мог уж он тревожить
Сердца кокеток записных!
Как рано мог уж он тревожить
Сердца кокеток записных!
Как рано мог уж он тревожить
Сердца кокеток записных!
Как рано мог уж он тревожить
Сердца кокеток записных!
Как рано мог уж он тревожить
Сердца кокеток записных!
Как рано мог уж он тревожить
Сердца кокеток записных!
Как рано мог уж он тревожить
Сердца кокеток записных!
Как рано мог уж он тревожить
Сердца кокеток записных!
Как рано мог уж он тревожить
Сердца кокеток записных!
Как рано мог уж он тревожить
Сердца кокеток записных!
Как рано мог уж он тревожить
Се

Модель скатывается в повторы и это плохо. Есть еще несколько параметров, которые помогают бороться с повторами.

### repetition_penalty
Этот параметр просто штрафует повторы. Чем он выше, тем сильнее штраф.

In [52]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5, top_k=50,
                        max_length=300,
                        repetition_penalty=3.5
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1",
  "use_cache": false
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Дождь идет 
С севера на юг,
Вдали Италии своей.

Как рано мог уж он тревожить
Сердца кокеток записных!
Когда ж хотелось уничтожить
Ему соперников своих,
Как он язвительно злословил!
Какие сети им готовил!
Но вы, блаженные мужья,
С ним оставались вы друзья:
Его ласкал супруг лукавый,
Его ревновал коварный,
Его забавлял жеманник,
Его забавляла ревность,
Его забавляла лень,
Его забавляла скука,
Его забавляла тоска,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его забавляла лень,
Его

### no_repeat_ngram_size
А этот параметр напрямую говорит, что нграммы такого размера не должны повторяться совсем

In [63]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5, top_k=100,
                        max_length=300,
                        repetition_penalty=3.5,
                        no_repeat_ngram_size=2,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1",
  "use_cache": false
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Дождь идет 
Вслед за мной по пятам,
Слегка за шалости бранил
И в Летний сад гулять водил.

Как рано мог уж он тревожить
Сердца кокеток записных!
Когда же юности мятежной
Пришла Евгению пора —
Пора надежд и грусти нежной…
Высокой страсти не имел он
Без принужденья в разговоре
Подслушать сердца первый звук:
«Да!" или: «Нет!»
Шутя невинность, он думал о ком-то еще;
Но вот судьба свела его с Жанной д’Арк. 

Они были любовницы самых нежных лет Европы. Онегин был, по мненью многих современников, «самый счастливый из людей всех времен и народов». Его считали одним из самых красивых молодых людей своего времени. Он блистал умом, красноречием, но холодной расчетливостью. Ему недоставало скромности в одежде, кротости в манерах, а главное — твердости характера. Поэты воспел Евгения в своем произведении «Руслан и Людмила», посвященном роману А. С. Пушкина «Евгений де Мопассан» (М., 1826), и дал несколько советов по стилю изложения романа. Евгений считал, что роман должен начинаться со слов «да» и

Результат не очень хороший, но это самая маленькая модель.